# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
MODEL_GEMINI = 'gemini-3-flash-preview'

In [6]:
# set up environment
gemini_api_key = os.getenv('GEMINI_API_KEY')

if not gemini_api_key:
      print("API key not found")
else:
      print("API key found and good so far")

API key found and good so far


In [ ]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

system_prompt= '''
Act as a friendly technical teacher. Simplify complex coding and engineering topics using this format:

Simple Summary: One sentence in plain English for a non-programmer.

Everyday Example: A real-life analogy (e.g., an API is like a restaurant waiter).

Easy Steps: A short, numbered list of how it works without jargon.

Clear Code: A brief snippet with descriptive variable names and helpful comments.

Why It Matters: One sentence on its real-world importance"
'''

In [ ]:

question

In [ ]:
system_prompt

In [33]:
def message_technical_gemini(user_prompt):

      gemini_client = OpenAI(
            base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
            api_key = gemini_api_key
      )

      messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
      ]

      response = gemini_client.chat.completions.create(
            model = MODEL_GEMINI,
            messages = messages
      )

      return response.choices[0].message.content

In [ ]:
message_technical_gemini("What is Html ?")

## Making the UI of the model using Gradio

In [27]:
import gradio as gr

In [28]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [ ]:
message_input = gr.Textbox(label="Technical question : ", info="Enter the question that you want to ask ? ", lines=7)
message_output = gr.Markdown(label="Response : ")

view = gr.Interface(
    fn=message_technical_gemini,
    title="GEMINI - gemini-3-flash-preview", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain what is HTML ?",
        "Explain the code to check whether a number is even or odd in Java ?",
        ], 
    js=force_dark_mode,
    flagging_mode="never"
    )
view.launch(share=True)

## Now making the stream UI for this.

In [40]:
def message_technical_gemini_stream(user_prompt):

      gemini_client = OpenAI(
            base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
            api_key = gemini_api_key
      )

      messages = [
            {"role":"system", "content": system_prompt},
            {"role":"user", "content": user_prompt}
      ]

      stream = gemini_client.chat.completions.create(
            model = MODEL_GEMINI,
            messages = messages,
            stream=True
      )

      result = ""
      for chunk in stream:
            result += chunk.choices[0].delta.content or ""
            yield result

In [ ]:
message_input = gr.Textbox(label="Technical question : ", info="Enter the question that you want to ask ? ", lines=7)
message_output = gr.Markdown(label="Response : ")

view = gr.Interface(
    fn=message_technical_gemini_stream,
    title="GEMINI - gemini-3-flash-preview", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain what is HTML ?",
        "Explain the code to check whether a number is even or odd in Java ?",
        ], 
    js=force_dark_mode,
    flagging_mode="never"
    )
view.launch(share=True)